# 🌐 WorldLoom Quick Start

**Unified interface for latent world models (DreamerV3, TD-MPC2)**

[![GitHub](https://img.shields.io/badge/GitHub-worldloom-blue?style=flat&logo=github)](https://github.com/yoshihyoda/worldloom)
[![License: MIT](https://img.shields.io/badge/License-MIT-yellow.svg)](https://opensource.org/licenses/MIT)

This notebook demonstrates:
1. Installation & Setup
2. Creating World Models (one-liner API)
3. Encoding & Decoding Observations
4. Imagination Rollouts (the core feature!)
5. Simple Training Demo
6. Visualization

## 1. Installation

Install WorldLoom directly from GitHub:

In [ ]:
# Install WorldLoom with training dependencies
!pip install -q "worldloom[training,viz] @ git+https://github.com/yoshihyoda/worldloom.git"

# Verify installation
import worldloom

print(f"WorldLoom version: {worldloom.__version__}")

In [ ]:
# Standard imports
import matplotlib.pyplot as plt
import torch

# WorldLoom imports
from worldloom import create_world_model, list_models

# Check GPU availability
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using device: {device}")

## 2. Available Models

WorldLoom provides a unified interface for multiple world model architectures:

In [ ]:
# List all available model presets
models = list_models(verbose=True)

print("Available World Models:")
print("=" * 60)
for name, info in models.items():
    print(f"\n{name}")
    print(f"  Params: {info['params']}")
    print(f"  Type: {info['type']}")
    print(f"  Description: {info['description']}")

## 3. Create a World Model (One-Liner!)

The `create_world_model()` function provides a simple, unified interface:

In [ ]:
# Create a DreamerV3 model for image observations
# obs_shape: (channels, height, width) for images
dreamer = create_world_model(
    "dreamerv3:size12m",
    obs_shape=(3, 64, 64),  # RGB images
    action_dim=4,  # 4 discrete actions
    device=device,
)

print("DreamerV3 Model:")
print(f"  Parameters: {sum(p.numel() for p in dreamer.parameters()):,}")
print(f"  Obs shape: {dreamer.config.obs_shape}")
print(f"  Action dim: {dreamer.config.action_dim}")

In [ ]:
# Create a TD-MPC2 model for state observations
# Perfect for continuous control tasks like MuJoCo
tdmpc = create_world_model(
    "tdmpc2:5m",
    obs_shape=(39,),  # State vector (e.g., HalfCheetah has 39-dim state)
    action_dim=6,  # 6 continuous actions
    device=device,
)

print("TD-MPC2 Model:")
print(f"  Parameters: {sum(p.numel() for p in tdmpc.parameters()):,}")
print(f"  Obs shape: {tdmpc.config.obs_shape}")
print(f"  Action dim: {tdmpc.config.action_dim}")

## 4. Encode & Decode Observations

World models learn a compressed latent representation of observations:

In [ ]:
# Create a random observation (batch of 2 images)
batch_size = 2
obs = torch.randn(batch_size, 3, 64, 64, device=device)

print(f"Input observation shape: {obs.shape}")

# Encode: observation -> latent state
with torch.no_grad():
    state = dreamer.encode(obs)

print("\nLatent state components:")
print(f"  Deterministic (h): {state.deterministic.shape}")
print(f"  Stochastic (z): {state.stochastic.shape}")
print(f"  Combined features: {state.features.shape}")

In [ ]:
# Decode: latent state -> predictions
with torch.no_grad():
    predictions = dreamer.decode(state)

print("Decoded predictions:")
for key, value in predictions.items():
    if isinstance(value, torch.Tensor):
        print(f"  {key}: {value.shape}")
    else:
        print(f"  {key}: {value}")

## 5. Imagination Rollouts 🚀

The core feature of world models: **predict future states without environment interaction!**

This enables:
- Planning ahead without costly environment steps
- Learning from imagined experience
- Model-based RL algorithms

In [ ]:
# Start from an initial observation
initial_obs = torch.randn(1, 3, 64, 64, device=device)

# Encode to get initial state
with torch.no_grad():
    initial_state = dreamer.encode(initial_obs)

# Define a sequence of actions (horizon=15 steps)
horizon = 15
action_dim = dreamer.config.action_dim

# Random action sequence: [horizon, batch_size, action_dim]
actions = torch.randn(horizon, 1, action_dim, device=device)

print(f"Initial state features: {initial_state.features.shape}")
print(f"Action sequence shape: {actions.shape}")

In [ ]:
# Run imagination rollout!
with torch.no_grad():
    trajectory = dreamer.imagine(initial_state, actions)

print("Imagination Trajectory:")
print(f"  Number of states: {len(trajectory.states)}")
print(f"  Predicted rewards shape: {trajectory.rewards.shape}")
print(f"  Continue probabilities shape: {trajectory.continues.shape}")

# Access individual predicted states
print(f"\n  State at t=0: features {trajectory.states[0].features.shape}")
print(f"  State at t=14: features {trajectory.states[-1].features.shape}")

In [ ]:
# Visualize predicted rewards over the imagination horizon
plt.figure(figsize=(10, 4))

rewards = trajectory.rewards.cpu().numpy().flatten()
continues = trajectory.continues.cpu().numpy().flatten()

plt.subplot(1, 2, 1)
plt.plot(rewards, marker="o", linewidth=2, markersize=4)
plt.xlabel("Time Step")
plt.ylabel("Predicted Reward")
plt.title("Imagined Rewards")
plt.grid(True, alpha=0.3)

plt.subplot(1, 2, 2)
plt.plot(continues, marker="s", linewidth=2, markersize=4, color="green")
plt.xlabel("Time Step")
plt.ylabel("Continue Probability")
plt.title("Episode Continuation")
plt.grid(True, alpha=0.3)
plt.ylim([0, 1.1])

plt.tight_layout()
plt.show()

## 6. Step-by-Step Prediction

You can also make single-step predictions with `predict()` and `observe()`:

In [ ]:
# predict(): Prior prediction (imagination, no observation)
action = torch.randn(1, action_dim, device=device)

with torch.no_grad():
    # Predict next state without seeing actual observation
    next_state_prior = dreamer.predict(initial_state, action)

print("Prior prediction (imagination):")
print(f"  Deterministic: {next_state_prior.deterministic.shape}")
print(f"  Stochastic: {next_state_prior.stochastic.shape}")

In [ ]:
# observe(): Posterior update (with actual observation)
next_obs = torch.randn(1, 3, 64, 64, device=device)  # Real observation

with torch.no_grad():
    # Update state with actual observation (posterior)
    next_state_posterior = dreamer.observe(initial_state, action, next_obs)

print("Posterior update (with observation):")
print(f"  Deterministic: {next_state_posterior.deterministic.shape}")
print(f"  Stochastic: {next_state_posterior.stochastic.shape}")

# The posterior is more accurate because it incorporates the actual observation
diff = (next_state_prior.stochastic - next_state_posterior.stochastic).abs().mean()
print(f"\nPrior vs Posterior stochastic difference: {diff:.4f}")

## 7. Training Demo

WorldLoom provides a complete training infrastructure:

In [ ]:
from worldloom.training import Trainer, TrainingConfig
from worldloom.training.data import create_random_buffer

# Create a fresh model for training
model = create_world_model(
    "dreamerv3:size12m",
    obs_shape=(3, 64, 64),
    action_dim=4,
    device=device,
)

# Create a replay buffer with random data for demo
# In practice, you'd collect real environment data
buffer = create_random_buffer(
    capacity=5000,
    obs_shape=(3, 64, 64),
    action_dim=4,
    num_episodes=50,
    episode_length=100,
    seed=42,
)

print(f"Buffer size: {len(buffer)} transitions")
print(f"Number of episodes: {buffer.num_episodes}")

In [ ]:
# Configure training (reduced for demo)
config = TrainingConfig(
    total_steps=500,  # Quick demo - use 50000+ for real training
    batch_size=8,  # Small batch for Colab
    sequence_length=16,  # Shorter sequences
    learning_rate=3e-4,
    log_every=100,
)

# Create trainer
trainer = Trainer(model, config)

print("Training configuration:")
print(f"  Total steps: {config.total_steps}")
print(f"  Batch size: {config.batch_size}")
print(f"  Sequence length: {config.sequence_length}")

In [ ]:
# Train!
print("Starting training...\n")
history = trainer.train(buffer)

print("\nTraining complete!")
print(f"Final loss: {history['loss'][-1]:.4f}")

In [ ]:
# Visualize training curves
plt.figure(figsize=(12, 4))

plt.subplot(1, 3, 1)
plt.plot(history["loss"])
plt.xlabel("Step")
plt.ylabel("Total Loss")
plt.title("Training Loss")
plt.grid(True, alpha=0.3)

if "reconstruction_loss" in history:
    plt.subplot(1, 3, 2)
    plt.plot(history["reconstruction_loss"], label="Reconstruction")
    plt.xlabel("Step")
    plt.ylabel("Loss")
    plt.title("Reconstruction Loss")
    plt.grid(True, alpha=0.3)

if "kl_loss" in history:
    plt.subplot(1, 3, 3)
    plt.plot(history["kl_loss"], color="orange")
    plt.xlabel("Step")
    plt.ylabel("KL Divergence")
    plt.title("KL Loss")
    plt.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## 8. Model Comparison: DreamerV3 vs TD-MPC2

| Feature | DreamerV3 | TD-MPC2 |
|---------|-----------|----------|
| **Input Type** | Images or State | State |
| **Latent Space** | Categorical (discrete) | SimNorm (continuous) |
| **Decoder** | Yes (reconstructs obs) | No (implicit model) |
| **Best For** | Atari, visual tasks | MuJoCo, robotics |
| **Planning** | Policy rollouts | MPC with Q-function |

In [ ]:
# Compare latent state structures
print("DreamerV3 State Structure:")
dreamer_obs = torch.randn(1, 3, 64, 64, device=device)
with torch.no_grad():
    dreamer_state = dreamer.encode(dreamer_obs)
print(f"  Deterministic (RSSM hidden): {dreamer_state.deterministic.shape}")
print(f"  Stochastic (categorical): {dreamer_state.stochastic.shape}")
print("  Has decoder: Yes")

print("\nTD-MPC2 State Structure:")
tdmpc_obs = torch.randn(1, 39, device=device)
with torch.no_grad():
    tdmpc_state = tdmpc.encode(tdmpc_obs)
print(f"  Deterministic (SimNorm): {tdmpc_state.deterministic.shape}")
print(f"  Stochastic: {tdmpc_state.stochastic}")
print("  Has decoder: No (implicit model)")

## 9. Save & Load Models

In [ ]:
import os

# Save the trained model
save_path = "./my_world_model"
model.save_pretrained(save_path)
print(f"Model saved to: {save_path}")

# List saved files
print("\nSaved files:")
for f in os.listdir(save_path):
    print(f"  {f}")

In [ ]:
# Load the saved model
loaded_model = create_world_model(save_path, device=device)
print(f"Loaded model type: {type(loaded_model).__name__}")
print(f"Loaded model config: {loaded_model.config}")

## 10. Next Steps

Congratulations! You've learned the basics of WorldLoom. Here are some next steps:

### Learn More
- [Full Documentation](https://worldloom.readthedocs.io)
- [GitHub Repository](https://github.com/yoshihyoda/worldloom)
- [Examples Directory](https://github.com/yoshihyoda/worldloom/tree/main/examples)

### Try These
1. **Train on real data**: Collect trajectories from Atari or MuJoCo
2. **Use larger models**: Try `dreamerv3:size50m` or `tdmpc2:48m`
3. **Build an agent**: Use imagination for planning

### Example Commands
```bash
# Collect Atari data
python examples/collect_atari.py --env Breakout --episodes 100

# Train DreamerV3
python examples/train_atari_dreamer.py --data atari_data.npz --steps 100000

# Visualize imagination
python examples/visualize_imagination.py --model ./outputs/model
```

---

**Questions or Issues?** Open an issue on [GitHub](https://github.com/yoshihyoda/worldloom/issues)

**Want to contribute?** Check out our [Contributing Guide](https://github.com/yoshihyoda/worldloom/blob/main/CONTRIBUTING.md)